In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import redis
import pickle
r = redis.Redis()
r.set('experiment', pickle.dumps([]))

True

In [4]:
import sys
sys.path.append('..')
from researchlib.single_import import *

Using GPU 0: Tesla V100-SXM2-16GB-N
Driver: 418.40.04
Researchlib version 19.08.1
Image version: 19.08.1
Current version is up-to-date!
 * Serving Flask app "researchlib.frontend.dashboard" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off

* Dashboard is open at http://<ip>:8050


In [5]:
train_loader = VisionDataset(vision.CIFAR10, batch_size=256, train=True)
test_loader = VisionDataset(vision.CIFAR10, batch_size=256, train=False)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
model = builder([
    AutoConvNet2d(3, 15, 'residual', filters=(64, 512), pooling_freq=4, flatten=False, se=False, sn=False, sd=True),
    nn.AdaptiveMaxPool2d(1),
    nn.Flatten(),
    nn.Linear(512, 10),
    nn.Softmax(-1)
])

3 64
64 64
64 64
64 64
64 128
128 128
128 128
128 128
128 256
256 256
256 256
256 256
256 512
512 512
512 512
512 512


In [7]:
runner = Runner(model, train_loader, test_loader, 'nesterov', 'nl', monitor_state='acc', monitor_mode='max', swa=False)

In [8]:
runner.init_model('default')

In [9]:
runner.preprocessing([Scaler([-1, 1])]).augmentation([HFlip(), Zoom()]).fit(2, lr=1e-2)

In [10]:
runner.report()

Experiment: default2019-08-14_08:51:38.109626
Checkpoints are saved in ./checkpoint/default2019-08-14_08:51:38.109626


Epoch,train_loss,train_acc,val_loss,val_acc,saved
1,2.033383,0.24522,1.551363,0.4162,*
2,1.750842,0.36172,1.411103,0.4866,*
